In [ ]:
# Cell 1
#1a_ae_rf_fusion_mix_training.ipynb

%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

# Setup path
current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

print(f"Project Root: {root_dir}")

# Import modules
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils
from sklearn.model_selection import train_test_split

# Kiểm tra đường dẫn Data Folder thay vì Data Path
print(f"Checking Data Directories:")
print(f"  - 2017: {config.DIR_2017} -> {'✅ Có' if config.DIR_2017.exists() else '❌ Không thấy'}")
print(f"  - 2018: {config.DIR_2018} -> {'✅ Có' if config.DIR_2018.exists() else '❌ Không thấy'}")

if not config.DIR_2017.exists() or not config.DIR_2018.exists():
    print("⚠️ CẢNH BÁO: Hãy kiểm tra lại đường dẫn folder trong src/config.py")

# --- KHỞI TẠO EXPERIMENT MỚI ---
# Bước này sẽ tạo folder dạng: results/experiments/exp_20251212_064500/
exp_paths = utils.setup_experiment_folder()
utils.log_experiment_details(exp_path=exp_paths['root'])
print(f"🚀 Experiment ID initialized.")

In [ ]:
# Cell 2: Load Datasets Separately
import numpy as np

try:
    print("🚀 Loading 2017 and 2018 datasets separately...")
    
    # Nạp 2017
    X_17, y_17 = preprocessing.load_single_dataset_year('2017', binary_mode=True)
    print(f"✅ Loaded 2017: {X_17.shape}")
    
    # Nạp 2018
    X_18, y_18 = preprocessing.load_single_dataset_year('2018', binary_mode=True)
    print(f"✅ Loaded 2018: {X_18.shape}")

except Exception as e:
    print(f"❌ Lỗi khi nạp dữ liệu: {e}")

In [ ]:
# Cell: Sanity Check - Kiểm tra Chuẩn hóa và Thứ tự
import pandas as pd
import numpy as np
from src import config, preprocessing

def verify_feature_order_and_normalization():
    print("🕵️♂️ BẮT ĐẦU KIỂM TRA TÍNH TOÀN VẸN DỮ LIỆU...\n")
    
    # 1. Kiểm tra Thứ tự (Order Guarantee)
    # ---------------------------------------------------------
    print("1️⃣  Kiểm tra Config (Khuôn mẫu):")
    expected_cols = config.SELECTED_FEATURES
    print(f"   - Số lượng features trong Config: {len(expected_cols)}")
    print(f"   - 5 features đầu: {expected_cols[:5]}")
    print(f"   - 5 features cuối: {expected_cols[-5:]}")
    print("   ✅ Kết luận: Biến X (numpy) của bạn CHẮC CHẮN tuân theo thứ tự này.")
    print("      (Vì code load dùng lệnh: chunk[config.SELECTED_FEATURES])\n")

    # 2. Kiểm tra Chuẩn hóa trên File Thực (Real Data Test)
    # ---------------------------------------------------------
    print("2️⃣  Kiểm tra thực tế trên file CSV (Mỗi năm 1 file):")
    
    # Lấy 1 file mẫu từ mỗi năm
    files_to_check = {
        "2017": next(config.DIR_2017.glob("*.csv"), None),
        "2018": next(config.DIR_2018.glob("*.csv"), None)
    }

    for year, file_path in files_to_check.items():
        if not file_path:
            print(f"   ⚠️ Không tìm thấy file {year}")
            continue
            
        print(f"\n   👉 Kiểm tra dataset {year}: {file_path.name}")
        
        # Đọc thô 5 dòng
        df_raw = pd.read_csv(file_path, nrows=5)
        print(f"      - Cột gốc (Raw): {df_raw.columns.tolist()[:3]} ...")
        
        # Áp dụng hàm chuẩn hóa
        df_norm = preprocessing.normalize_column_names(df_raw)
        
        # Kiểm tra xem các cột quan trọng có xuất hiện không
        # Ví dụ: Kiểm tra xem cột 'Destination Port' (đã chuẩn hóa) có mặt không
        target_col = "Destination Port"
        if target_col in df_norm.columns:
            print(f"      - ✅ Đã tìm thấy '{target_col}' (Chuẩn hóa OK)")
        else:
            print(f"      - ❌ CẢNH BÁO: Không thấy '{target_col}'. Có thể mapping tên bị sai!")
            
        # Kiểm tra sự khớp lệnh với Config
        # Tạo df giả lập bước lọc
        for col in config.SELECTED_FEATURES:
            if col not in df_norm.columns:
                df_norm[col] = 0.0 # Giả lập fill missing
        
        df_final = df_norm[config.SELECTED_FEATURES]
        
        # So sánh thứ tự
        if df_final.columns.tolist() == config.SELECTED_FEATURES:
            print(f"      - ✅ Thứ tự cột sau khi lọc khớp 100% với Config.")
        else:
            print(f"      - ❌ SAI THỨ TỰ! Cần kiểm tra lại code.")

verify_feature_order_and_normalization()

In [ ]:
# Cell 3: Independent Splitting & Mixed Training Construction
from sklearn.model_selection import train_test_split

print("✂️ Splitting 2017 and 2018 independently (80/20)...")

# 1. Chia 2017
X_17_train, X_17_test, y_17_train, y_17_test = train_test_split(
    X_17, y_17, test_size=0.2, random_state=config.SEED, stratify=y_17
)

# 2. Chia 2018
X_18_train, X_18_test, y_18_train, y_18_test = train_test_split(
    X_18, y_18, test_size=0.2, random_state=config.SEED, stratify=y_18
)

# 3. Gộp các phần Training (Scenario 2)
X_train = np.vstack([X_17_train, X_18_train])
y_train = np.concatenate([y_17_train, y_18_train])

# 4. Giữ các phần Test riêng biệt để đánh giá đa chiều
# X_test và y_test tổng hợp (nếu vẫn muốn đánh giá chung)
X_test_all = np.vstack([X_17_test, X_18_test])
y_test_all = np.concatenate([y_17_test, y_18_test])

print(f"📊 Mixed Train Size: {X_train.shape}")
print(f"📊 Test 2017 Size:  {X_17_test.shape}")
print(f"📊 Test 2018 Size:  {X_18_test.shape}")

# Xóa dữ liệu thô để giải phóng RAM
del X_17, X_18, X_17_train, X_18_train
import gc; gc.collect()

In [ ]:
# Cell 4: Scaling & Save Scaler
print("🚀 Scaling data...")

# 1. Khởi tạo Scaler
scaler = preprocessing.get_scaler()

# 2. Fit và Transform trên tập MIXED TRAIN
# Đây là bước quan trọng nhất: Scaler chỉ được học thông số từ tập huấn luyện [cite: 514]
X_train_scaled = scaler.fit_transform(X_train)

# 3. Transform các tập Test độc lập (Sử dụng thông số của tập Train) 
# Chúng ta scale riêng từng bộ để phục vụ đánh giá đa chiều ở Cell 10
X_17_test_scaled = scaler.transform(X_17_test)
X_18_test_scaled = scaler.transform(X_18_test)
X_test_scaled    = scaler.transform(X_test_all) # Tập test tổng hợp (nếu cần)

print("✅ Done scaling for all datasets.")

# --- LƯU SCALER VÀO FOLDER EXPERIMENT ---
import joblib
print("💾 Saving scaler...")
scaler_path = exp_paths["models"] / "scaler.joblib"
joblib.dump(scaler, scaler_path)
print(f"✅ Saved scaler to: {scaler_path}")

In [ ]:
# Cell 5: Train Autoencoder (L=30, N=5)
print(f"🚀 Training Autoencoder on: {config.DEVICE}")

# 1. Định nghĩa L và N dựa trên kết quả tối ưu
L = 30  # Input dimension cho AE 
N = 5   # Latent dimension (điểm ngọt) [cite: 525]

# 2. CẮT DỮ LIỆU: Chỉ lấy 30 đặc trưng mRMR đầu tiên
# X_train_scaled đang có 65 cột, chúng ta lấy [:, :30]
X_train_ae_input = X_train_scaled[:, :L]

# 3. Khởi tạo model với input_dim khớp với dữ liệu đã cắt
ae_model = autoencoder.DeepAutoencoder(
    input_dim=L,             # Phải là 30 
    latent_dim=N,            # Phải là 5 [cite: 525]
    hidden_layers=[22, 12]   # Cấu trúc nén dần phù hợp cho L=30 
)

ae_save_path = exp_paths["models"] / "ae_model.pth"

# 4. Huấn luyện với dữ liệu đã cắt (X_train_ae_input)
ae_model = autoencoder.train_ae(ae_model, X_train_ae_input, save_path=ae_save_path)

print("✅ AE training done with L=30, N=5.")

In [ ]:
# Cell 6: Extract Latent Features Z
print("🚀 Extracting Latent Features (Z) ...")

# Định nghĩa L khớp với tham số lúc Train AE
L = 30 

# Trích xuất cho tập Train (Cần cắt lấy 30 cột đầu)
X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled[:, :L])

# Trích xuất cho các tập Test độc lập (Cũng cần cắt lấy 30 cột đầu)
X_17_test_latent = autoencoder.extract_features(ae_model, X_17_test_scaled[:, :L])
X_18_test_latent = autoencoder.extract_features(ae_model, X_18_test_scaled[:, :L])
X_test_all_latent = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

print(f"✅ Original input shape for AE: (N, {L})")
print(f"✅ Latent features extracted shape: {X_train_latent.shape}") # Sẽ là (N, 5) nếu N_latent=5

In [ ]:
import gc

# 1. Xóa các biến dữ liệu cũ để giải phóng RAM
print("🗑️ Cleaning up RAM...")
try:
    del X
    del X_train
    del X_test
    # Nếu có các biến phụ khác thì xóa nốt
except:
    pass

# 2. Ép Python thu hồi bộ nhớ ngay lập tức
gc.collect()

print("✅ RAM Cleaned.")

In [ ]:
# Cell 7: Chuẩn bị dữ liệu cho Fusion

# 1. Lấy danh sách tên cột đầy đủ
all_feature_names = config.SELECTED_FEATURES # List 65 cột

# 2. Tìm vị trí index của 20 cột mRMR trong list 65 cột
try:
    mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]
    print(f"✅ Found indices for {len(mrmr_indices)} mRMR features.")
except ValueError as e:
    print(f"❌ Error: Feature name not found in SELECTED_FEATURES. Check config.py!\nDetails: {e}")
    raise e

# 3. Cắt dữ liệu (Slicing) từ tập đã Scale
# Chỉ lấy các cột nằm trong mrmr_indices
print("✂️ Slicing mRMR features from Scaled Data...")
X_train_mrmr = X_train_scaled[:, mrmr_indices]
X_test_mrmr  = X_test_scaled[:, mrmr_indices]

print(f"X_train_mrmr shape: {X_train_mrmr.shape}") # MONG ĐỢI: (N, 20)

# (Lưu ý: Bạn đã extract latent ở Cell 6 rồi, không cần chạy lại dòng extract ở đây nữa để tiết kiệm thời gian,
# nhưng nếu bạn muốn chắc chắn thì cứ để nguyên dòng extract ở dưới cũng không sao)
# print("🧠 Extracting Latent Features...")
# X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled)
# X_test_latent  = autoencoder.extract_features(ae_model, X_test_scaled)

In [ ]:
# Cell 8: Fusion Strategy [mRMR 20] + [Latent 5]
import numpy as np

print("🔗 Fusing: [mRMR 20] + [Latent 5] ...")

# Lấy 20 đặc trưng mRMR đầu tiên làm Direct Features
mrmr_top = 20 

# Fusion cho tập Train
X_train_fusion = np.hstack([X_train_scaled[:, :mrmr_top], X_train_latent])

# Fusion cho các tập Test (Sử dụng các biến latent đã trích xuất ở Cell 6)
X_17_test_fusion = np.hstack([X_17_test_scaled[:, :mrmr_top], X_17_test_latent])
X_18_test_fusion = np.hstack([X_18_test_scaled[:, :mrmr_top], X_18_test_latent])
X_test_all_fusion = np.hstack([X_test_scaled[:, :mrmr_top], X_test_all_latent]) # Đổi tên cho khớp Cell 10

print(f"🔥 Fusion Completed. Shapes:")
print(f"   - Train Fusion: {X_train_fusion.shape}")
print(f"   - Test All Fusion: {X_test_all_fusion.shape}")

In [ ]:
utils.log_experiment_details(exp_path=exp_paths['root'])


In [ ]:
# Cell 9: Train RF on Fusion Data
print("🚀 Starting RF Training on Fusion Data...")

rf_save_path = exp_paths["models"] / "rf_model.joblib"

rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)

print("RF training done!")

In [ ]:
# Cell 10: Multi-Scenario Evaluation
print("\n--- STARTING FINAL EVALUATION ---")

test_scenarios = [
    (X_17_test_fusion, y_17_test, "Unseen 2017 (Legacy)"),
    (X_18_test_fusion, y_18_test, "Unseen 2018 (Modern)"),
    (X_test_all_fusion, y_test_all, "Global Mixed Test")
]

for X_t, y_t, name in test_scenarios:
    print(f"\n🚀 Evaluating Scenario: {name}")
    evaluation.evaluate_model(
        model=rf_model, 
        X_test=X_t, 
        y_test=y_t, 
        save_dir=exp_paths["figures"], 
        dataset_name=name
    )